In [5]:
%pip install pandas 
%pip install requests


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 648.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 4.6 MB/s eta 0:00:03
   - -------------------------------------- 0.4/11.6 MB 5.3 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.6 MB 5.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.6 MB 4.8 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/11.6 MB 4.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/11.6 MB 5.8 MB/s eta 0:00:02
   ------ --------------------------------- 2.0/11.6 MB 6.8 MB/s eta 0:00:02
   -------- ------------------------------- 2.5/11.6 MB 7.0 MB/s eta 0:00:02
   ------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ------------------ --------------------- 30.7/64.9 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 64.9/64.9 kB 868.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/167.3 kB ? eta -:--:--
   ------- -------------------------------- 30.7/167.3 kB 1.4 MB/s eta 0:00:01
   -------------- ------------------------ 61.4/167.3 kB 825.8 kB/s eta 0:00:01
   ------------------------- ------------ 112.6/167.3 kB 939.4 kB/s eta 0:00:01
   ------------------------------------ --- 153.6/167.3 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- 167.3/167.3 kB 918.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/101.8 kB ? eta -:--:--
   ------------------------ --------------- 61.4/101.8 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 101.8/101.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/70.4 kB ? eta -:--:--


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


##### Importando bibliotecas

In [1]:
import requests
import pandas as pd
import sqlite3

##### Criando funções

In [ ]:
def extrair_dados(api_url):
    resposta = requests.get(api_url)
    if resposta.status_code == 200:
        return resposta.json() 
    else:
        print(f'Erro ao acessar a API {api_url}')
        return None 

def salvar_no_banco(df, tabela_nome):
    print(f"Salvando dados na tabela {tabela_nome}...")
    df.to_sql(tabela_nome, conn, if_exists='replace', index=False)

def validar_tabelas(data_base, tabelas):
    conn = sqlite3.connect(data_base)
    for tabela in tabelas:
        query = f"SELECT COUNT(*) AS total FROM {tabela}"
        total = pd.read_sql(query, conn).iloc[0]['total']
        if total == 0:
            print(f"⚠️ Atenção: A tabela '{tabela}' está vazia!")
        else:
            print(f"✅ Tabela '{tabela}' validada com sucesso. Total de registros: {total}")
    conn.close()

def validar_tipos_dados(df, colunas_tipos):
    for coluna, tipo in colunas_tipos.items():
        if not df[coluna].map(type).eq(tipo).all():
            print(f"⚠️ Erro: A coluna '{coluna}' não possui todos os valores como {tipo}.")
        else:
            print(f"✅ A coluna '{coluna}' possui todos os valores como {tipo}.")


##### Criando tabelas

In [ ]:
pokemons = extrair_dados('https://pokeapi.co/api/v2/pokemon?offset=0&limit=20')
berrys = extrair_dados('https://pokeapi.co/api/v2/berry?offset=0&limit=20')
locations = extrair_dados('https://pokeapi.co/api/v2/location?offset=0&limit=20')

tabela_pokemons = pd.DataFrame(pokemons)
tabela_berrys = pd.DataFrame(berrys)
tabela_locations = pd.DataFrame(locations)

#tratando dados 


detalhes_pokemons = []

for pokemon in tabela_pokemons['results']:
    detalhes = extrair_dados(pokemon['url'])
    detalhes_pokemons.append({
        'name': detalhes['name'],
        'id': detalhes['id'],
        'height': detalhes['height'],
        'weight': detalhes['weight']
    })

detalhes_berrys = []

for berry in tabela_berrys['results']:
    detalhes = extrair_dados(berry['url'])
    detalhes_berrys.append({
        'name': detalhes['name'],
        'id': detalhes['id'],
        'growth_time': detalhes['growth_time'],
        'max_harvest': detalhes['max_harvest'],
        'natural_gift_power': detalhes['natural_gift_power'],
        'size': detalhes['size'],
        'smoothness': detalhes['smoothness'],
        'soil_dryness': detalhes['soil_dryness']
    })

detalhes_locations = []

for location in tabela_locations['results']:
    detalhes = extrair_dados(location['url'])
    detalhes_locations.append({
        'name': detalhes['name'],
        'region': detalhes['region']     
    })


pokemons_basic_data = pd.DataFrame(detalhes_pokemons)

berrys_basic_data = pd.DataFrame(detalhes_berrys)

location_basic_data = pd.DataFrame(detalhes_locations)


##### Tratando os dados

In [83]:
# tabela 1 - pokemons
pokemons_basic_data = pokemons_basic_data.dropna().drop_duplicates()
pokemons_basic_data['name'] = pokemons_basic_data['name'].astype(str)
pokemons_basic_data['id'] = pokemons_basic_data['id'].astype(int)
pokemons_basic_data['height'] = pokemons_basic_data['height'].astype(float)
pokemons_basic_data['weight'] = pokemons_basic_data['weight'].astype(float)

# tabela 2 - berry
berrys_basic_data = berrys_basic_data.dropna().drop_duplicates()
berrys_basic_data['name'] = berrys_basic_data['name'].astype(str)
berrys_basic_data['id'] = berrys_basic_data['id'].astype(int)
berrys_basic_data['growth_time'] = berrys_basic_data['growth_time'].astype(float)
berrys_basic_data['max_harvest'] = berrys_basic_data['max_harvest'].astype(float)
berrys_basic_data['natural_gift_power'] = berrys_basic_data['natural_gift_power'].astype(float)
berrys_basic_data['size'] = berrys_basic_data['size'].astype(float)
berrys_basic_data['smoothness'] = berrys_basic_data['smoothness'].astype(float)
berrys_basic_data['soil_dryness'] = berrys_basic_data['soil_dryness'].astype(float)

# tabela 3 - location
location_basic_data['region'] = location_basic_data['region'].apply(lambda x: x['name'])
location_basic_data = location_basic_data.dropna().drop_duplicates()
location_basic_data['name'] = location_basic_data['name'].astype(str)
location_basic_data['region'] = location_basic_data['region'].astype(str)


In [84]:
validar_tipos_dados(pokemons_basic_data,{'name': str})
validar_tipos_dados(berrys_basic_data,{'name': str})
validar_tipos_dados(location_basic_data,{'name': str})

✅ A coluna 'name' possui todos os valores como <class 'str'>.
✅ A coluna 'name' possui todos os valores como <class 'str'>.
✅ A coluna 'name' possui todos os valores como <class 'str'>.


#### Conectar e criar banco de dados 

In [85]:

conn = sqlite3.connect('pokemon.db')

salvar_no_banco(pokemons_basic_data, 'pokemons_basic_data')
salvar_no_banco(berrys_basic_data, 'berrys_basic_data')
salvar_no_banco(location_basic_data, 'location_basic_data')

conn.close()

Salvando dados na tabela pokemons_basic_data...
Salvando dados na tabela berrys_basic_data...
Salvando dados na tabela location_basic_data...


In [86]:
validar_tabelas('pokemon.db', ['pokemons_basic_data', 'berrys_basic_data', 'location_basic_data'])

Tabela 'pokemons_basic_data' validada com sucesso. Total de registros: 20
Tabela 'berrys_basic_data' validada com sucesso. Total de registros: 20
Tabela 'location_basic_data' validada com sucesso. Total de registros: 20
